**Decision Variables**

$f_i$ : 1 if table i  is used, 0 otherwise, where $i=1...N$

$g_{ij}:$ 1 if both table i and j are used, 0 otherwise,  

$x_i =$ bottom left x coordinate of table i

$y_i =$ bottom left x coordinate of table i

$l_{ij} =$ 1 if table i is located left to j

$b_{ij} =$ 1 if table i is located below j

**Parameters**

$d$ : The minimum physical distance required between tables

$c_i =$ max # people that can be seated @ table i, where $i=1...N$

$C$ = Minimum desired establishment capacity

$w_i =$ width of table i

$h_i =$ height of table i

$R_w =$ width of dining space

$R_h =$ height of dining space

$U_{ij}$ = Underperformance measure of staff when table i and j are used

**Objective Function:

$min$ $z$ = $g_1^+ + g_2^- +\sum_{i=1}^{N}\sum_{j=1, i\neq j}^{N}g_{ij}U_{ij}$

**s.t**
Force non-linear element to be 0 or 1:

$g_{ij} >= f_i+f_j-1 \ i, j \in T and \ i \neq j$

No overlap if the following constraint is true:

$l_{ij} + l_{ji} + b_{ij} + b_{ji} + (1-x_i) + (1-x_j) >= 1, \ i, j \in T, \ i<j$ 

If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate:

$x_i +w_i +d/2 <= x_j + W - W_{ij}, \ i, j \in T$

If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate:

$y_i +h_i+d/2 <= y_j + H - H_{ij}, \ i, j \in T$

Tables must exist within the dining space:

$x_i >= 0, \ i \in T$

$x_i+w_i+d/2 <= R_x, \ i \in T$

$y_i >= 0, \ i \in T$

$y_i+h_i+d/2 <= R_h, \ i \in T$

Unused table space must be under U:

$R_wR_h \ - \sum_{i=1}^{N}f_i(w_i+d/2)(h_i+d/2) - (g_1^+-g_1^-) = U, \ i \in T$ 

Facility capacity must be over C to remain profitable

$\sum_{i=1}^{N}c_if_i - (g_2^+-g_2^-) = C, \ i \in T$

Binary variable constraints:

$l_{ij}, b_{ij} \ \in {0,1}$

$f_{i} \ \in {0,1}$

$g_{ij}\in {0,1}$

#### Step 1: Import grobipy module

In [30]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


#### Step 2: Define the model

In [31]:
m = gp.Model()

#### Step 3: Define the sets and sizes

In [32]:
#number of tables
N = 2


#### Step 4: Define the parameters

In [33]:

    #capacity for each table
    #'c': [2, 8],
    
    #width for each table
    w= [2, 1]
    
    #height for each table
    h= [2,1]
    
    #Minimum physical distane required between tables
    d= 2
    
    #Width of dining space
    Rw= 2

    #Height of dining space
    Rh=2

    #Minimum desired establishment capacity
    #'C': 10,

    #Underperformance measure of staff when table i and j are used
    #'Uij': [[1,1,1,1], [1,1,1,1], [1,1,1,1], [1,1,1,1]]
    









#### Step 5: Define decision variables

In [34]:
#binary, if table i is used
f = {}
for i in range(N):
    f[i] = m.addVar(vtype=GRB.BINARY, obj = (w[i]*h[i]), name="f_" +str(i))
    
#TODO: binary, if both table i and j are used

#bottom left x coordinate of table i
x = {}
for i in range(N):
    x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))
    
#bottom left y coordinate of table i
y = {}
for i in range(N):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))
    
    
#binary, if table i is located left to table j
l = {}
for i in range(N):
    for j in range(N):
        if i != j:
            l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

#binary, if table i is located below to table j
b = {}
for i in range(N):
    for j in range(N):
        if i != j:
            b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

#### Step 6: Set the objective function

In [35]:
m.modelSense = GRB.MAXIMIZE

#### Step 7: Add constraints

In [36]:
# no overlap
m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)
# If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)
#If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)
#table space must exist within the dining space
m.addConstrs(x[i] >= 0 for i in range(N))
m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
m.addConstrs(y[i] >= 0 for i in range(N))
m.addConstrs(y[i]+h[i]<= Rh for i in range (N))
m.update()

#### Step 8: Solve the model

In [37]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 14 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x3d2ad934
Variable types: 4 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 14 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%


#### Insert variable values

In [38]:
print (h)
print (Rw)
print (Rh)
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
    

[2, 1]
2
2
f_0 1
f_1 0
x_0 0
x_1 0
y_0 0
y_1 0
l_01 0
l_10 0
b_01 0
b_10 0


In [39]:
df = pd.DataFrame(dataSet, columns=['c', 'w', 'h'])
display(df)

,c,w,h
0,NaN,"[2, 1]","[2, 1]"


In [42]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=[0,1,2,0,None,3,3,5,5,3], y=[0,2,0,0,None,0.5,1.5,1.5,0.5,0.5], fill="toself"))
fig.show()